In [1]:
import pdfplumber
import pandas as pd
from textblob import TextBlob

ModuleNotFoundError: No module named 'pdfplumber'

In [ ]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 64.8 MB/s eta 0:00:00


In [ ]:
esg_terms = {
    "Environmental": ["emission", "pollution", "waste", "carbon", "climate", "deforestation"],
    "Social": ["labor", "human rights", "discrimination", "diversity", "safety", "community"],
    "Governance": ["fine", "lawsuit", "corruption", "fraud", "regulation", "penalty"]
}

In [ ]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:  # some pages may be empty
                text += page_text + " "
    return text.lower()

In [ ]:
def analyze_report(company, year, pdf_path):
    text = extract_text_from_pdf(pdf_path)

    records = []
    total_score = 0
    for category, keywords in esg_terms.items():
        for word in keywords:
            count = text.count(word)
            if count > 0:
                # Sentiment score for the whole report (basic check)
                sentiment = TextBlob(text).sentiment.polarity
                total_score += count
                records.append([company, year, word, category, count, sentiment])

    return records, total_score

In [ ]:
all_records = []
company_scores = []

In [ ]:
reports = [
    ("HoneyWell", 2023, "/content/hon-esg-report.pdf"),
    ("Infosys", 2023, "/content/infosys-esg-report-2022-23.pdf"),
    ("Adani", 2023, "/content/latest report.pdf"),
    ("Wipro", 2023, "/content/wipro-sustainability-report-fy-2023-2024.pdf")
]

In [ ]:
for company, year, path in reports:
    records, score = analyze_report(company, year, path)
    all_records.extend(records)
    company_scores.append([company, year, score])

In [ ]:
df = pd.DataFrame(all_records, columns=["Company", "Year", "Risk_Term", "Category", "Frequency", "Sentiment_Score"])
df_scores = pd.DataFrame(company_scores, columns=["Company", "Year", "Overall_ESG_Risk_Score"])
final_df = pd.merge(df, df_scores, on=["Company", "Year"], how="left")
final_df.to_excel("esg_risk_data.xlsx", index=False)

print("✅ ESG dataset for 4 companies saved as 'esg_risk_data.xlsx'")

✅ ESG dataset for 4 companies saved as 'esg_risk_data.xlsx'
